# Exploration
We will first need to have playwright initialize itself so that we can programmatically crawl a website to automatically gather the reviews

In [2]:
!playwright install


BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu20.04-x64.
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu20.04-x64.
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu20.04-x64.
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu20.04-x64.
Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libicudata.so.66                                 ║
║     libicui18n.so.66                                 ║
║     libicuuc.so.66                                   ║
║     libjpeg.so.8                                     ║
║     libwebp.so.6                                     ║
║     libpcre.so.3                                     ║
║     libffi.so.7     

## Using Playwright within a jupyter notebook
Playwright will usually work in python scripts. To explore the pages, we'll need to use asyncio. You can view the full explanation in this stack overflow answer ["How to use the Playwright library in a Jupyter notebook instead of using a regular .py script (on Windows)"](https://stackoverflow.com/a/71702599)

In [3]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

await page.goto("http://whatsmyuseragent.org/")

<Response url='http://ww1.whatsmyuseragent.org/?usid=17&utid=34089031374' request=<Request url='http://ww1.whatsmyuseragent.org/?usid=17&utid=34089031374' method='GET'>>

## Accessing the website

I have chosen to collect reviews for LTT Store, as I follow their youtube and find their products to be quality. LTT Store used to have reviews embedded in the product page, but when I was drafting this notebook, it wasn't accessible. Fortunately, I had some old emails, and I was able to trace it back to judge.me. To keep it simple, let's open the backpack reviews [link](https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack)

In [74]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.firefox.launch(headless=False)
page = await browser.new_page()

# Set the viewport size
await page.set_viewport_size({"width": 1920, "height": 1080})

await page.goto("https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack")


<Response url='https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack' request=<Request url='https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack' method='GET'>>

You should see a window that looks like this
![](../dev-log/supporting_files/images/2024-08-18/20240818163226.png)
With the website loaded, we'll now want to filter the reviews by most recent first. To access this, we will need to inspect the elements and review the html. Within the playwright window, right click anywhere on the page.

In [62]:
await page.click("select.filter-sort__sort-dropdown")

# Wait for the options to be visible and then select 'Most Recent'
await page.select_option("select.filter-sort__sort-dropdown", label="Most Recent")

['[object Object]']

In [76]:
dropdown = page.locator('role=combobox')
await dropdown.wait_for(state="visible")
await dropdown.click()

# Select the option that triggers the network request
await dropdown.select_option(label="Most Recent")

['[object Object]']

In [64]:

    # Set up the response handler to log all responses
    def log_response(response):
        print("Response received:", response.url(), response.status())
    page.on("response", log_response)

    # Set the viewport size
    await page.set_viewport_size({"width": 1920, "height": 1080})

    # Navigate to the specific page
    await page.goto("https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack")

    # Handling dropdown interaction
    dropdown = page.locator('role=combobox')
    await dropdown.click()
    await dropdown.select_option(label="Most Recent")  # Assuming this triggers the API request

    # Wait for a specific response to finish
    response = await page.wait_for_response(lambda r: 'judge.me/api/v1/widgets/product_review' in r.url() and r.status() == 200)

    # Optionally, process the response
    json_body = await response.json()
    print("Specific response content:", json_body)

    # Keep the browser open long enough to observe manually or further automate
    await asyncio.sleep(10)  # Wait for 10 seconds

    # Close the browser
    # await browser.close()

TypeError: Locator.click: 'str' object is not callable

In [58]:
test = []

In [59]:
test

[]

In [60]:
    playwright = await async_playwright().start()
    browser = await playwright.firefox.launch(headless=False)
    page = await browser.new_page()

    # Set the viewport size
    await page.set_viewport_size({"width": 1920, "height": 1080})

    # https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=1&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&review_type=product-reviews&review_source=all_sources&verified_only=false

    # Listen for specific responses
    async def handle_response(response):
        url = response.url
        if "judge.me/api/v1/widgets/product_review" in url and 'sort_by=created_at' in url:
            print("Response received from:", url)
            print("Content-Type:", response.headers['content-type'])
            try:
                json_body = await response.json()
                print("Specific response content:", json_body)
                test.append(json_body)
            except Exception as e:
                print("Error processing JSON:", e)

    # Attach the response handler to the page
    page.on("response", handle_response)

    # Navigate to the specific page
    await page.goto("https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack")

    # Wait for the dropdown to be visible and interact with it
    dropdown = page.locator('role=combobox')
    await dropdown.wait_for(state="visible")
    await dropdown.click()

    # Select the option that triggers the network request
    await dropdown.select_option(label="Most Recent")

    # Optionally, hold the browser open a bit longer to ensure all responses are captured
    await asyncio.sleep(10)  # Adjust the sleep duration as necessary

Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=1&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&review_type=product-reviews&review_source=all_sources&verified_only=false
Content-Type: application/json; charset=utf-8
Specific response content: {'reviews': [{'uuid': 'b8f0ba8d-2fa5-4261-b48e-b6be38b4fcdb', 'title': 'A very nice backpack', 'rating': 4, 'body_html': "<p>Overall, the best backpack I've owned and will continue to use it well into the future. </p>\n\n<p>Pros:\n<br />Feels very sturdy and durable </p>\n\n<p>Lots of storage space </p>\n\n<p>Straps are quite comfortable </p>\n\n<p>Design is practical and well thought-out </p>\n\n<p>Water-resistance has been excellent so far</p>\n\n<p>Cons:\n<br />The foam shoulder padding on the straps seems to be wearing out very unevenly. I always have the backpack over bot

Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=2&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&search=&review_type=product-reviews&review_source=all_sources&verified_only=false
Content-Type: application/json; charset=utf-8
Specific response content: {'reviews': [{'uuid': 'ee3f743b-ab7e-4051-b6b4-c67d97e90fa0', 'title': 'Disappointed in backpack', 'rating': 2, 'body_html': '<p>On paper, this backpack reads and looks terrific and I purchased it as a birthday gift for my grandson (upon his advice). Unfortunately, after two outings, he found that the backpack "rode" on his shoulders in such a way as to cause both chaffing and discomfort.\n<br />He LOVES the concept of this backpack (lots of room, plan of backpack is superior to others)  and has tried to make adjustments to \n<br />how he wears it but has not been succ

In [63]:
test

[{'reviews': [{'uuid': 'b8f0ba8d-2fa5-4261-b48e-b6be38b4fcdb',
    'title': 'A very nice backpack',
    'rating': 4,
    'body_html': "<p>Overall, the best backpack I've owned and will continue to use it well into the future. </p>\n\n<p>Pros:\n<br />Feels very sturdy and durable </p>\n\n<p>Lots of storage space </p>\n\n<p>Straps are quite comfortable </p>\n\n<p>Design is practical and well thought-out </p>\n\n<p>Water-resistance has been excellent so far</p>\n\n<p>Cons:\n<br />The foam shoulder padding on the straps seems to be wearing out very unevenly. I always have the backpack over both shoulders, not sure why the left strap would flatten out while the right still looks new.\n<br />A bit heavy, even while empty</p>\n\n<p>Description advertised it stands up on its own but, I haven't found that to be reliable </p>\n\n<p>I wish it had some kind of reflective spots on the back for cyclists/scooter riders at night</p>\n\n<p>For my use case, a slightly bigger main-area, and a smaller sle

In [46]:
# hardcoded next page
next_page_button = page.locator("span.pagination__page-number:nth-child(11)")
        
# Wait for the button to be visible
await next_page_button.wait_for(state="visible")
        
# Click the 'Next Page' button
await next_page_button.click()

In [62]:
# Locate the 'Next Page' button by finding the span with the chevron_right icon
next_page_button = page.locator('span.pagination__page-number--icon:has-text("chevron_right")')

# Wait for the button to be visible
await next_page_button.wait_for(state="visible")

# Click the 'Next Page' button
await next_page_button.click()

# As a script

## v1

In [70]:
import asyncio
import json
from playwright.async_api import async_playwright

async def handle_response(response, output_file):
    url = response.url
    if "judge.me/api/v1/widgets/product_review" in url and 'sort_by=created_at' in url:
        print("Response received from:", url)
        print("Content-Type:", response.headers['content-type'])
        try:
            json_body = await response.json()
            # print("Specific response content:", json_body)
            
            # Create a new dictionary where the key is the URL and the value is the list of reviews
            reviews_data = {url: json_body['reviews']}
            
            # Prettify the JSON output
            prettified_json = json.dumps(reviews_data, indent=4)
            
            # Dump the prettified JSON data into the file
            with open(output_file, 'a') as f:
                f.write(prettified_json)
                f.write('\n')  # Add a newline for each JSON entry
        except Exception as e:
            print("Error processing JSON:", e)

async def main():
    async with async_playwright() as playwright:
        browser = await playwright.firefox.launch(headless=False)
        page = await browser.new_page()
        
        output_file = "backpack_reviews.json"
        max_pages = 299  # Set the maximum number of pages to process
        current_page = 1

        # Set the viewport size
        await page.set_viewport_size({"width": 1920, "height": 1080})

        # Attach the response handler to the page
        page.on("response", lambda response: handle_response(response, output_file))

        # Navigate to the specific page
        await page.goto("https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack")

        # Wait for the dropdown to be visible and interact with it
        dropdown = page.locator('role=combobox')
        await dropdown.wait_for(state="visible")
        await dropdown.click()

        # Select the option that triggers the network request
        await dropdown.select_option(label="Most Recent")

        while current_page <= max_pages:
            # Locate the 'Next Page' button by finding the span with the chevron_right icon
            next_page_button = page.locator('span.pagination__page-number--icon:has-text("chevron_right")')

            # Check if the 'Next Page' button is visible
            if await next_page_button.is_visible():
                # Click the 'Next Page' button
                await next_page_button.click()

                await asyncio.sleep(2)
                # Wait for the page to load
                await page.wait_for_load_state("networkidle")

                # Increment the page count
                current_page += 1
            else:
                # If the 'Next Page' button is not visible, we've reached the last page
                print("Reached the last page.")
                break

        # await browser.close()

# Run the main function
await(main())


Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=1&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&review_type=product-reviews&review_source=all_sources&verified_only=false
Content-Type: application/json; charset=utf-8
Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=2&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&search=&review_type=product-reviews&review_source=all_sources&verified_only=false
Content-Type: application/json; charset=utf-8
Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=3&sort_

TimeoutError: Timeout 30000ms exceeded.

## v2

In [73]:
import asyncio
import json
from playwright.async_api import async_playwright

async def handle_response(response, output_file):
    url = response.url
    if "judge.me/api/v1/widgets/product_review" in url and 'sort_by=created_at' in url:
        print("Response received from:", url)
        print("Content-Type:", response.headers['content-type'])
        try:
            json_body = await response.json()
            
            # Create a new dictionary where the key is the URL and the value is the list of reviews
            reviews_data = {url: json_body['reviews']}
            
            # Prettify the JSON output
            prettified_json = json.dumps(reviews_data, indent=4)
            
            # Dump the prettified JSON data into the file
            with open(output_file, 'a') as f:
                f.write(prettified_json)
                f.write('\n')  # Add a newline for each JSON entry
        except Exception as e:
            print("Error processing JSON:", e)

async def navigate_to_next_page(page, current_page, max_pages):
    try:
        # Locate the 'Next Page' button by finding the span with the chevron_right icon
        next_page_button = page.locator('span.pagination__page-number--icon:has-text("chevron_right")')

        # Check if the 'Next Page' button is visible
        if await next_page_button.is_visible():
            # Click the 'Next Page' button
            await next_page_button.click()

            await asyncio.sleep(2)
            # Wait for the page to load
            await page.wait_for_load_state("networkidle")
            return True  # Successfully navigated to the next page
        else:
            # If the 'Next Page' button is not visible, we've reached the last page
            print("Reached the last page.")
            return False  # No more pages to navigate to
    except Exception as e:
        print(f"Error during page navigation: {e}")
        return False

async def handle_network_idle_issue(page):
    try:
        # Navigate to the previous page
        previous_page_button = page.locator('span.pagination__page-number--icon:has-text("chevron_left")')
        if await previous_page_button.is_visible():
            await previous_page_button.click()

            # Wait for the specified time before resuming
            print("Waiting for 300 seconds before retrying...")
            await asyncio.sleep(300)
        else:
            print("No previous page button found.")
    except Exception as e:
        print(f"Error handling network idle issue: {e}")

async def main():
    async with async_playwright() as playwright:
        browser = await playwright.firefox.launch(headless=False)
        page = await browser.new_page()
        
        output_file = "backpack_reviews.json"
        max_pages = 299  # Set the maximum number of pages to process
        current_page = 1

        # Set the viewport size
        await page.set_viewport_size({"width": 1920, "height": 1080})

        # Attach the response handler to the page
        page.on("response", lambda response: handle_response(response, output_file))

        # Navigate to the specific page
        await page.goto("https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack")

        # Wait for the dropdown to be visible and interact with it
        dropdown = page.locator('role=combobox')
        await dropdown.wait_for(state="visible")
        await dropdown.click()

        # Select the option that triggers the network request
        await dropdown.select_option(label="Most Recent")

        while current_page <= max_pages:
            try:
                # Attempt to navigate to the next page
                success = await navigate_to_next_page(page, current_page, max_pages)
                
                if success:
                    # Increment the page count
                    current_page += 1
                else:
                    break
            except Exception as e:
                print(f"Error detected: {e}")
                await handle_network_idle_issue(page)
                
        # await browser.close()

# Run the main function
await(main())


Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=1&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&review_type=product-reviews&review_source=all_sources&verified_only=false
Content-Type: application/json; charset=utf-8
Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=2&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&search=&review_type=product-reviews&review_source=all_sources&verified_only=false
Content-Type: application/json; charset=utf-8
Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=3&sort_

## Trying with requests
Unfortunately my playwright function seems to break at page 50. While inspecting the network tab, I realized that the api was a simple http request
<blockquote>

### HTTP Request
GET https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=1&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&review_type=product-reviews&review_source=all_sources&verified_only=false

### HTTP Response
- **Status:** 200
- **Version:** HTTP/2
- **Transferred:** 4.77 kB (23.72 kB size)
- **Referrer Policy:** strict-origin-when-cross-origin
- **DNS Resolution:** System

### Headers

#### Response Headers
access-control-allow-origin: *  
cache-control: max-age=1200, public  
content-encoding: gzip  
content-type: application/json; charset=utf-8  
date: Sun, 18 Aug 2024 19:07:21 GMT  
etag: W/"be98ba27a582c82cb3e5e7cbd31f15a6"  
set-cookie: _judgeme_session=**generic_session_cookie**; path=/; expires=Sun, 01 Sep 2024 19:07:21 -0000; secure; HttpOnly; SameSite=None  
strict-transport-security: max-age=31536000  
vary: Accept-Encoding  
x-content-type-options: nosniff  
X-Firefox-Spdy: h2  
x-frame-options: SAMEORIGIN  
x-request-id: f694a9cb-20e8-4a84-961d-c0e2de4f564b  
x-runtime: 0.160691  
x-xss-protection: 1; mode=block

#### Request Headers
Accept: application/json, text/plain, */*  
Accept-Encoding: gzip, deflate, br, zstd  
Accept-Language: en-US,en;q=0.5  
Connection: keep-alive  
Cookie: current_country=**generic_country_cookie**; _judgeme_session=**generic_session_cookie**; intercom-id-z09y8g87=**generic_intercom_id**; intercom-session-z09y8g87=; intercom-device-id-z09y8g87=**generic_device_id**  
DNT: 1  
Host: judge.me  
Referer: https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack  
Sec-Fetch-Dest: empty  
Sec-Fetch-Mode: cors  
Sec-Fetch-Site: same-origin  
User-Agent: Mozilla/5.0 (Windows NT 10.0; rv:129.0) Gecko/20100101 Firefox/129.0  
X-JUDGEME-REDESIGN: true
</blockquote>

In [13]:
import requests

# Define the URL and parameters for the GET request
url = "https://judge.me/api/v1/widgets/product_review"
params = {
    'platform': 'shopify',
    'shop_domain': 'linus-tech-tips-store.myshopify.com',
    'api_token': 'snmsgMp9hQgfed2xliHdvcJig1o',
    'external_id': '6649895256167',
    'page': '300',
    'sort_by': 'created_at',
    'sort_dir': 'desc',
    'json_request': 'true',
    'per_page': '10',  # Number of items per page set to 1000
    'review_type': 'product-reviews',
    'review_source': 'all_sources',
    'verified_only': 'false'
}

# Send the GET request
response = requests.get(url, params=params)

# Print the status code and the content of the response
print("Status Code:", response.status_code)
print("Response Content:", response.text)


Status Code: 200
Response Content: {"reviews":[{"uuid":"b30cfca0-0043-499c-bfab-f7a404f2a22b","title":"The last backpack you will buy","rating":5,"body_html":"\u003cp\u003eVery high quality bag. Everything just feels nice like the zippers etc. fits perfectly on the body and so much more comfortable than other bags. It has so many features that other bags are missing. Multiple sleeves for laptop, portable monitor, tablet. Water bottle holder on the inside which I like a lot. A separate large compartment that can hold clothes or whatever else. And my favourite feature is that it stands upright on its own so I don’t need to prop it up against something when I set it down.\u003c/p\u003e","body":"Very high quality bag. Everything just feels nice like the zippers etc. fits perfectly on the body and so much more comfortable than other bags. It has so many features that other bags are missing. Multiple sleeves for laptop, portable monitor, tablet. Water bottle holder on the inside which I like

In [9]:
import requests
import json
import time

output_file = "backpack_reviews_test.json"

def write_json(data, url, file, is_last):
    with open(file, 'a') as f:
        # Write the URL and start the JSON array for reviews
        f.write(f'"{url}": ')
        json.dump(data, f, indent=4)
        if not is_last:
            f.write(',\n')  # Add a comma if it's not the last item
        else:
            f.write('\n')  # No comma at the end

# Initialize the JSON file with an opening brace
with open(output_file, 'w') as f:
    f.write('{\n')

total_pages = 2  # Set the total number of pages you expect to fetch
current_page = 0  # Initialize the current page counter

for page in range(1, total_pages + 1):
    url = f"https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page={page}&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&review_type=product-reviews&review_source=all_sources&verified_only=false"
    response = requests.get(url)
    current_page += 1  # Increment the current page counter
    is_last = current_page == total_pages  # Check if this is the last page

    if response.status_code == 200:
        json_body = response.json()
        write_json(json_body['reviews'], response.url, output_file, is_last)
        print(f"Processed page: {page}")
    else:
        print(f"Failed to fetch page: {page}")

    time.sleep(2)

# Finalize the JSON file with a closing brace
with open(output_file, 'a') as f:
    f.write('}\n')

print("JSON file created successfully.")


Processed page: 1
Processed page: 2
JSON file created successfully.
